<a href="https://colab.research.google.com/github/matthewvilaysack/Fractals/blob/main/PeterBenitez(peterjb3)_MatthewVilaysack(mvilay02)_CS224V_fall2025_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS224V Fall2025 HW1

**⚠️[IMPORTANT] Make a copy of this notebook. DO NOT EDIT THIS NOTEBOOK**

1. Run the code block below to download starter code
2. Navigate to google colab content directory (i.e. 📁 icon on the left side bar) and click the refresh/reload button


In [ ]:
!pip install -U gdown
!gdown https://drive.google.com/uc?id=1wUpcOYU-eT1pe_je6qQLa8v4TBPUhPae
!unzip code.zip
!rm  -r __MACOSX
!mv cs224v_hw1_code/* .
!rm -r cs224v_hw1_code
!rm requirements.txt
!rm notebook.ipynb

Downloading...
From: https://drive.google.com/uc?id=1wUpcOYU-eT1pe_je6qQLa8v4TBPUhPae
To: /content/code.zip
100% 377k/377k [00:00<00:00, 123MB/s]
Archive:  code.zip
   creating: cs224v_hw1_code/
  inflating: __MACOSX/._cs224v_hw1_code  
  inflating: cs224v_hw1_code/requirements.txt  
  inflating: __MACOSX/cs224v_hw1_code/._requirements.txt  
  inflating: cs224v_hw1_code/notebook.ipynb  
  inflating: __MACOSX/cs224v_hw1_code/._notebook.ipynb  
  inflating: cs224v_hw1_code/.env    
  inflating: __MACOSX/cs224v_hw1_code/._.env  
   creating: cs224v_hw1_code/data/
  inflating: __MACOSX/cs224v_hw1_code/._data  
  inflating: cs224v_hw1_code/create_submission.py  
  inflating: __MACOSX/cs224v_hw1_code/._create_submission.py  
   creating: cs224v_hw1_code/src/
  inflating: __MACOSX/cs224v_hw1_code/._src  
  inflating: cs224v_hw1_code/data/action_item_5_database_exploration_precomputed.json  
  inflating: __MACOSX/cs224v_hw1_code/data/._action_item_5_database_exploration_precomputed.json  
  in

run the code block below to install dependencies

In [ ]:
!pip install dspy-ai==3.0.3
!pip install crawl4ai==0.7.4
!pip install langchain_text_splitters==0.3.11
!pip install jupyter==1.0.0
!pip install nbconvert[webpdf]

!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
173.7 MiB [] 0% 132.2s173.7 MiB [] 0% 25.2s173.7 MiB [] 0% 20.5s173.7 MiB [] 0% 9.9s173.7 MiB [] 1% 5.8s173.7 MiB [] 1% 4.5s173.7 MiB [] 2% 3.7s173.7 MiB [] 3% 3.1s173.7 MiB [] 4% 2.9s173.7 MiB [] 5% 2.7s173.7 MiB [] 5% 3.0s173.7 MiB [] 6% 2.7s173.7 MiB [] 7% 2.6s173.7 MiB [] 8% 2.5s173.7 MiB [] 8% 2.6s173.7 MiB [] 9% 2.6s173.7 MiB [] 10% 2.5s173.7 MiB [] 11% 2.3s173.7 MiB [] 12% 2.2s173.7 MiB [] 13% 2.2s173.7 MiB [] 15% 2.0s173.7 MiB [] 16% 1.9s173.7 MiB [] 17% 1.8s173.7 MiB [] 18% 1.8s173.7 MiB [] 20% 1.7s173.7 MiB [] 21% 1.7s173.7 MiB [] 22% 1.6s173.7 MiB [] 23% 1.5s173.7 MiB [] 24% 1.5s173.7 MiB [] 25% 1.4s173.7 MiB [] 26% 1.5s173.7 MiB [] 27% 1.5s173.7 MiB [] 28% 1.4s173.7 MiB [] 29% 1.4s173.7 MiB [] 30% 1.4s173.7 MiB [] 32% 1.3s173.7 MiB [] 33% 1.3s

In [ ]:
import json
import os
from typing import List, Tuple

import dspy
import httpx
from tqdm import tqdm

from src.dataclass import RetrievedDocument, LiteratureSearchAgentResponse, LiteratureSearchAgentRequest
from src.encoder import Encoder
from src.literature_search import LiteratureSearchAgent
from src.lm import init_lm, LanguageModelProviderConfig, LanguageModelProvider, LiteLLMServerConfig
from src.retriever_agent.serper_rm import SerperRM
from src.rag import RagAgent
from src.dataclass import RagResponse, RagRequest

from google.colab import userdata

In [ ]:
TOPIC = "Evolving Military Strategies in the Russia-Ukraine War and Future Implications"

ACLED_DB_DESCRIPTION = """ACLED is a global conflict and event data repository. It captures and analyzes conflicts worldwide, from local conflicts in Africa to international armed conflicts. It includes events like civil wars, military operations, and terrorist attacks. Acled's data helps monitor conflicts, understand conflict dynamics, and inform policy decisions."""

## Action Item 1: LLM API Configuration

> **Reference:** See **Section 2 (Background) → Action Item 1** in the handout for complete setup instructions and theoretical background.

### Setup Requirements

| Step | Task | Implementation Details |
|------|------|------------------------|
| 1 | **Obtain API Key** | Access the provided portal at [cs224v-litellm-portal.genie.stanford.edu](http://cs224v-litellm-portal.genie.stanford.edu) |
| 2 | **Configure Secrets** | Add `LITELLM_API_KEY` and `LITELLM_API_BASE` to secrets |
| 3 | **Verify Configuration** | Execute the validation code block below |

### Note

Please find the "🔑" shape icon on the left sidebar. Configure secrets key and values there. Remember to toggle "Notebook access"

---

In [ ]:
## Action item 1.
test_lm_config = LanguageModelProviderConfig(
      provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
      model_name="gpt-4.1-mini",
      temperature=0.0,
      max_tokens=10,
      litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
  )
test_lm = init_lm(test_lm_config)
print(test_lm("say 'Hello!' as is")[0]) # Expect to see "Hello!" or something similar

Hello!


In [ ]:
encoder = Encoder(model_name="text-embedding-3-small", **{"api_key": userdata.get("LITELLM_API_KEY"), "api_base": userdata.get("LITELLM_API_BASE")})
embedding = await encoder.aencode("hello")
assert len(embedding[0]) == 1536
print(f"✅ encoder is working")

✅ encoder is working


## Action Item 2: Web Retrieval Service Configuration

> **Reference:** See **Section 3.1 (RAG Systems) → Action Item 2** in the handout for detailed background on web retrieval architectures and alternative search providers.

### Serper.dev Setup Process

The Retrieval-Augmented Generation pipeline requires a web search backend for information retrieval. This implementation uses Serper.dev as the search provider.

**Configuration Steps:**
1. **API Key Acquisition:** Register at [serper.dev](https://serper.dev) to obtain 2,500 free search credits
2. **Secret Configuration:** Add `SERPER_API_KEY` to Google Colab Secrets with "Notebook access" enabled
3. **System Validation:** Execute the test code block below

### Expected Output
Successful configuration should produce:
- Confirmation message: "✅ retriever is working"
- Example document structure demonstrating the retrieved data format

### Technical Context
This setup establishes the foundation for the RAG pipeline's retrieval component. The retrieved document structure includes URL sources, content snippets, and metadata that will be processed through subsequent pipeline stages including content extraction, chunking, and semantic reranking.

---

In [ ]:
serper_retriever = SerperRM(api_key=userdata.get("SERPER_API_KEY"), encoder=encoder)
retrieved_document: RetrievedDocument = await serper_retriever.aretrieve("stanford new AI research")
assert len(retrieved_document) > 0
print(f"✅ retriever is working")
print(f"example output")
print(json.dumps(retrieved_document[0].to_dict(), indent=2))

[FETCH]... ↓ https://storm.genie.stanford.edu/                                                                    |
✓ | ⏱: 19.11s 

[SCRAPE].. ◆ https://storm.genie.stanford.edu/                                                                    |
✓ | ⏱: 0.15s 

[COMPLETE] ● https://storm.genie.stanford.edu/                                                                    |
✓ | ⏱: 19.35s 

[FETCH]... ↓ https://www.pantaos.com/en/post/ein-sturm-zieht-auf-stanford-s-neues-ki-recherche-tool-1             |
✓ | ⏱: 19.83s 

[SCRAPE].. ◆ https://www.pantaos.com/en/post/ein-sturm-zieht-auf-stanford-s-neues-ki-recherche-tool-1             |
✓ | ⏱: 1.48s 

...

[COMPLETE] ● https://setr.stanford.edu/technology/artificial-intelligence/2025                                    |
✓ | ⏱: 35.07s 

[FETCH]... ↓ https://epoch.ai/                                                                                    |
✓ | ⏱: 37.00s 

[SCRAPE].. ◆ https://epoch.ai/                                                                                    |
✓ | ⏱: 0.71s 

[COMPLETE] ● https://epoch.ai/                                                                                    |
✓ | ⏱: 37.73s 

✅ retriever is working
example output
{
  "url": "https://news.stanford.edu/artificial-intelligence",
  "excerpts": [
    "## Stanford Report cookie usage information\nWe want to provide stories, announcements, events, leadership messages and resources that are relevant to you. Your selection is stored in a browser cookie which you can remove at any time by visiting the \"Show me...\" menu at the top right of the page. For more, read our .\nA new study reveals the subtle effects of survey interfaces on people\u2019s responses \u2013 and how those small differences can add up.\nScholars across campus are leveraging AI to drive remarkable advancements in fields from robotics to neuroscience to mining, while fostering a cautious approach to the application of the technology.\nStanford researchers explored how humans and generative AI interpret color. What they found wasn\u2019t so black and white.\nDeveloped at Stanford Medicine, Nuclei.io is designed to improve workflow and diagnosis in 

## Action Item 3: End-to-End RAG System Evaluation

> **Reference:** See **Section 3.1 (RAG Systems) → Action Item 3** in the handout for comprehensive RAG pipeline theory and component analysis.

### Objective
Evaluate the complete Retrieval-Augmented Generation pipeline to understand information flow through the following stages:

```
Query → Internet Search → Content Extraction → Document Chunking → Semantic Reranking → Answer Generation
```

### Step 1: Basic Pipeline Test
Execute the code block below to test the RAG system with a basic query. The system will:
- Process the input question through the complete pipeline
- Save results to `output/action_item_3_rag_response.json`
- Demonstrate the integration of retrieval and generation components

### Analysis Requirements
Review the output format to understand:
- Retrieved document structure and source attribution
- Semantic reranking results and relevance scoring
- Generated answer quality and citation methodology

---

In [ ]:
# configure the language model for RAG agent
rag_lm_config = LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-4.1-mini",
    temperature=1.0,
    max_tokens=10000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
)
rag_lm = init_lm(rag_lm_config)

# initialize the RAG agent
rag = RagAgent(retriever=serper_retriever, rag_lm=rag_lm)

# forward the request to the RAG agent
rag_response: RagResponse = await rag.aforward(RagRequest(question="What is the latest news on AI research at Stanford?", max_retriever_calls=1))

# make output directory if it doesn't exist
os.makedirs("output", exist_ok=True)

# save the response to a file
with open("output/action_item_3_rag_response.json", "w") as f:
    json.dump(rag_response.to_dict(), f, indent=2)

print(f"✅ Result saved to output/action_item_3_rag_response.json")


[FETCH]... ↓ https://med.stanford.edu/news/topics/artificial-intelligence.html                                    |
✓ | ⏱: 4.07s 

[SCRAPE].. ◆ https://med.stanford.edu/news/topics/artificial-intelligence.html                                    |
✓ | ⏱: 0.06s 

[COMPLETE] ● https://med.stanford.edu/news/topics/artificial-intelligence.html                                    |
✓ | ⏱: 4.16s 

[FETCH]... ↓ https://www.techdirt.com/2025/10/02/stanford-stu...ted-workslop-actually-making-productivity-worse/  |
✓ | ⏱: 8.99s 

[SCRAPE].. ◆ https://www.techdirt.com/2025/10/02/stanford-stu...ted-workslop-actually-making-productivity-worse/  |
✓ | ⏱: 4.32s 

...

[COMPLETE] ● https://www.forbes.com/sites/rashishrivastava/20...e-maths-hardest-problems-and-create-harder-ones/  |
✓ | ⏱: 33.83s 

[FETCH]... ↓ https://www.pcmag.com/news/ai-workslop-is-plaguing-american-companies-says-stanford-research         |
✓ | ⏱: 52.21s 

[SCRAPE].. ◆ https://www.pcmag.com/news/ai-workslop-is-plaguing-american-companies-says-stanford-research         |
✓ | ⏱: 1.39s 

[COMPLETE] ● https://www.pcmag.com/news/ai-workslop-is-plaguing-american-companies-says-stanford-research         |
✓ | ⏱: 53.65s 

✅ Result saved to output/action_item_3_rag_response.json


### Step 2: Advanced Query Testing

> **Reference:** Section 3.1 emphasizes the importance of evaluating RAG systems across diverse query types to assess robustness and performance characteristics.

#### Test Case A: Recency-Focused Query (`rag_question_recency`)
**Objective:** Evaluate the system's capability to retrieve and synthesize recent information.

**Requirements:**
- Formulate a factual question requiring current information for `rag_question_recency` (e.g., "What are the latest products from Apple in 2025?")
- **Note:** Do not use the provided example; create an original query
- Focus on testing temporal information retrieval accuracy

#### Evaluation Criteria

For this test question, document and analyze:
- **Quality of retrieved sources** (`rag_question_recency_quality_comment`, `rag_question_depth_quality_comment`): List all unique URLs from the RAG results one line per URL followed by [yes] or [no, {reason}] to indicate whether the URL is of high quality and should be included and cited when generating the final answer.
- **Relevance retrieved information** (`rag_question_recency_relevance_comment`, `rag_question_depth_relevance_comment`): Do all retrieved documents contain information that directly answers the question? Take a look at uncited documents, make comment on whether you think they are irrelevant to the question and thus LLM didn't cited it.
- **Comprehensiveness of generated answer** (`rag_question_recency_comprehensiveness_comment`, `rag_question_depth_comprehensiveness_comment`): Does the final answer synthesize information well and provide a complete response? Does LLM fully make use of all retrieved information? What are details that are left not included in the final answer.

---

In [ ]:
rag_question_recency = "Where did natural disasters occur in 2025 in the USA?"

rag_response: RagResponse = await rag.aforward(RagRequest(question=rag_question_recency, max_retriever_calls=1))

# save the response to a file
with open("output/action_item_3_rag_response_recency.json", "w") as f:
    json.dump(rag_response.to_dict(), f, indent=2)

print(f"✅ Result saved to output/action_item_3_rag_response_recency.json")

[FETCH]... ↓ https://en.wikipedia.org/wiki/Category:2025_natural_disasters_in_the_United_States                   |
✓ | ⏱: 5.01s 

[SCRAPE].. ◆ https://en.wikipedia.org/wiki/Category:2025_natural_disasters_in_the_United_States                   |
✓ | ⏱: 1.32s 

[COMPLETE] ● https://en.wikipedia.org/wiki/Category:2025_natural_disasters_in_the_United_States                   |
✓ | ⏱: 6.41s 

[FETCH]... ↓ https://stormtrack.org/threads/natural-disaster-losses-so-far-in-2025.33174/                         |
✓ | ⏱: 6.44s 

[SCRAPE].. ◆ https://stormtrack.org/threads/natural-disaster-losses-so-far-in-2025.33174/                         |
✓ | ⏱: 2.46s 

...

[COMPLETE] ● https://www.munichre.com/en/company/media-relati...25/natural-disaster-figures-first-half-2025.html  |
✓ | ⏱: 26.57s 

[FETCH]... ↓ https://www.youtube.com/watch?v=zlS1BaBgg3s                                                          |
✓ | ⏱: 28.80s 

[SCRAPE].. ◆ https://www.youtube.com/watch?v=zlS1BaBgg3s                                                          |
✓ | ⏱: 0.27s 

[COMPLETE] ● https://www.youtube.com/watch?v=zlS1BaBgg3s                                                          |
✓ | ⏱: 29.08s 

✅ Result saved to output/action_item_3_rag_response_recency.json


In [ ]:
rag_question_recency_quality_comment = """
- https://yaleclimateconnections.org/2025/07/u-s-socked-with-15-billion-dollar-weather-disasters-during-the-1st-half-of-2025/ [yes]
- https://disasterphilanthropy.org/disasters/2025-us-tornadoes-and-severe-storms/?utm_source=chatgpt.com [yes]
- https://worldpopulationreview.com/state-rankings/natural-disasters-by-state [yes]
- https://www.munichre.com/en/company/media-relations/media-information-and-corporate-news/media-information/2025/natural-disaster-figures-first-half-2025.html [yes]
- https://ncdp.columbia.edu/us-natural-hazards-and-climate-change/ [no, {it contains only projections and historical context and not facts about actual 2025 events}]
"""


rag_question_recency_relevance_comment = """
- https://yaleclimateconnections.org/2025/07/u-s-socked-with-15-billion-dollar-weather-disasters-during-the-1st-half-of-2025/ [directly relevant, gives detailed accounts of U.S. 2025 wildfires (Palisades / Eaton Fire), storms, and tornadoes with dates and loss figures]
- https://disasterphilanthropy.org/disasters/2025-us-tornadoes-and-severe-storms/ [directly relevant, covers regional floods in Texas, New Mexico, New Jersey, and New York, and tornadoes in Arkansas with impact and fatalities]
- https://worldpopulationreview.com/state-rankings/natural-disasters-by-state [directly relevant, shows which states are most disaster prone in 2025 quantitatively with FEMA and NOAA data]
- https://www.munichre.com/en/company/media-relations/media-information-and-corporate-news/media-information/2025/natural-disaster-figures-first-half-2025.html [somewhat relevant, emphasizes U.S. losses (LA fires and storms) but focuses on high-level financial overviews rather than locations]
- https://ncdp.columbia.edu/us-natural-hazards-and-climate-change/ [irrelevant, provides climate risk projections and preparedness insights but not where actual 2025 disaster events occurred]

So no, not all articles are directly relevant to answering this question.
"""


rag_question_recency_comprehensiveness_comment = """
- Does the final answer synthesize information well and provide a complete response?
  [yes, it synthesizes information well and gives a pretty complete response for where disasters occurred in the US in 2025]

- Does LLM fully make use of all retrieved information?
  [yes, it uses the cited and retrieved sources effectively; uncited ones are either somewhat relevant or correctly irrelevant: broadly uses (Munich Re) and correctly leaves out (NCDP)]

- What are details that are left not included in the final answer?
  [the answer left out humanitarian aspects such as specific death counts, displacement maps/figures, and local details beyond bigger states like CA/TX]
"""



with open("output/action_item_3_rag_response_recency_comment.json", "w") as f:
    json.dump({
        "quality_comment": rag_question_recency_quality_comment,
        "relevance_comment": rag_question_recency_relevance_comment,
        "comprehensiveness_comment": rag_question_recency_comprehensiveness_comment
    }, f, indent=2)
print(f"✅ Result saved to output/action_item_3_rag_response_recency_comment.json")

✅ Result saved to output/action_item_3_rag_response_recency_comment.json


#### Test Case B: Technical Depth Query (`rag_question_depth`)
**Objective:** Assess the system's performance on specialized technical topics requiring domain expertise.

**Requirements:**
- Design a question addressing a niche technical domain for `rag_question_depth` (e.g., "How do transformer attention mechanisms handle positional encoding?")
- **Note:** Do not use the provided example; formulate an original technical query
- Evaluate the system's ability to retrieve and synthesize specialized knowledge

**Apply the same evaluation framework as detailed above for comprehensive analysis.**

---

In [ ]:
rag_question_depth = "How can blocked (tiled) matrix multiplication be optimized for transformer architectures?"

rag_response: RagResponse = await rag.aforward(RagRequest(question=rag_question_depth, max_retriever_calls=1))

# save the response to a file
with open("output/action_item_3_rag_response_depth.json", "w") as f:
    json.dump(rag_response.to_dict(), f, indent=2)

print(f"✅ Result saved to output/action_item_3_rag_response_depth.json")

[FETCH]... ↓ https://arxiv.org/pdf/2503.16731                                                                     |
✓ | ⏱: 3.67s 

[SCRAPE].. ◆ https://arxiv.org/pdf/2503.16731                                                                     |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://arxiv.org/pdf/2503.16731                                                                     |
✓ | ⏱: 3.74s 

[FETCH]... ↓ https://www.reddit.com/r/learnprogramming/commen...ve/how_does_blocking_increase_speedup_of_matrix/  |
✓ | ⏱: 4.68s 

[SCRAPE].. ◆ https://www.reddit.com/r/learnprogramming/commen...ve/how_does_blocking_increase_speedup_of_matrix/  |
✓ | ⏱: 0.12s 

...

[COMPLETE] ● https://astralord.github.io/posts/transformer-inference-optimization-toolset/                        |
✓ | ⏱: 48.01s 

[FETCH]... ↓ https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html     |
✓ | ⏱: 48.52s 

[SCRAPE].. ◆ https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html     |
✓ | ⏱: 0.43s 

[COMPLETE] ● https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html     |
✓ | ⏱: 48.96s 

✅ Result saved to output/action_item_3_rag_response_depth.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rag_question_depth_quality_comment = """
  - https://arxiv.org/html/2503.16731v3 [yes]
	- https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/nki/tutorials/matrix_multiplication.html [yes]
	- https://alvinwan.com/how-to-tile-matrix-multiplication/ [yes]
	- https://stackoverflow.com/questions/15829223/loop-tiling-blocking-for-large-dense-matrix-multiplication [no, {this is a stackoverflow post which is a forum for users to ask and answer questions; although there are quite a few (N=10) replies in the reply section, this is not an indicator of high credibility or many posts on the forum since it is anonymous and people may not tell the turth, hence this source is of lower quality than the others}]
	- https://burn.dev/blog/sota-multiplatform-matmul/ [yes]
	- https://massedcompute.com/faq-answers/?question=What%20are%20the%20most%20efficient%20matrix%20multiplication%20algorithms%20for%20large-scale%20transformer-based%20models? [no, {the description states that this is AI generated content, which should be viewed with a cautionary and skeptical eye. }]
"""
rag_question_depth_relevance_comment = """
- All of the high quality technical sources (first 4) directly answer the question of "How does blocked (tiled) matrix multiplication optimize transformer architectures?" but the lower quality sources such as the StackOverflow post does not directly answer the question (in fact, it was posted 12 years ago, when the Attention is All You Need paper did not exist yet about transformer architecture.
--> the other low quality source MassedCompute does in fact directly answer the question (answers "What are the most efficient matrix multiplication algorithms for large-scale transformer-based models?")
"""
rag_question_depth_comprehensiveness_comment = """
- Does the final answer synthesize information well and provide a complete response?
-> Yes, the final answer synthesizes multiple tiling methods such as two level tiling, parallel computation of tiling, blocking dims., and hardware aware scheduling and ties them to transformer workloads.

- Does LLM fully make use of all retrieved information?
-> Yes, the final answer does make use of all the information from all the sources for the technical question, even the less credible ones.

- What are details that are left not included in the final answer?
-> There were some ommitted points about key benchmarks for performance such as GFLOPS for throughput, memory bandwidth savings, and the practicality of choosing tile sizes of tile dimensions. More details about flash attention with IO awareness would have rounded out a better holistic picture of the answer.
"""


with open("output/action_item_3_rag_response_depth_comment.json", "w") as f:
    json.dump({
        "quality_comment": rag_question_depth_quality_comment,
        "relevance_comment": rag_question_depth_relevance_comment,
        "comprehensiveness_comment": rag_question_depth_comprehensiveness_comment
    }, f, indent=2)
print(f"✅ Result saved to output/action_item_3_rag_response_depth_comment.json")

✅ Result saved to output/action_item_3_rag_response_depth_comment.json


## Action Item 4: Autonomous Literature Search Evaluation

> **Reference:** See **Section 3.2 (Autonomous Literature Search)** in the handout for comprehensive coverage of deep research paradigms, trajectory tracking, and autonomous exploration strategies.

### Objective
Evaluate the autonomous literature search system using a complex, multi-dimensional research topic that requires systematic exploration and evidence synthesis.

**Research Topic:** *"Evolving Military Strategies in the Russia-Ukraine War and Future Implications"*

### Step 1: System Execution
Execute the literature search agent using the code block below. The system will:
- Autonomously decompose the research topic into focused sub-queries
- Maintain an exploration trajectory across multiple search iterations
- Synthesize findings into a comprehensive research summary

**Output Location:** `output/action_item_4_literature_search_response.json`

---

In [ ]:
literature_search_planning_lm = init_lm(LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-4.1", # NOTE: planning invovles intelligence, so we use a more powerful model.
    temperature=1.0,
    max_tokens=10000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
))

answer_synthesis_lm = init_lm(LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-5-mini", # NOTE: synthesis does not require high intelligence, but requires minimal hallucination. GPT-5-mini is a good balance.
    temperature=1.0,
    max_tokens=20000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
))

# initialize the literature search agent
literature_search_agent = LiteratureSearchAgent(rag_agent=rag, literature_search_lm=literature_search_planning_lm, answer_synthesis_lm=answer_synthesis_lm)

# run the literature search agent
literature_search_response: LiteratureSearchAgentResponse = await literature_search_agent.aforward(LiteratureSearchAgentRequest(topic=TOPIC))

# save the response to a file
with open("output/action_item_4_literature_search_response.json", "w") as f:
    json.dump(literature_search_response.to_dict(), f, indent=2)

print(f"✅ Result saved to output/action_item_4_literature_search_response.json")

Starting literature search for topic: Evolving Military Strategies in the Russia-Ukraine War and Future Implications
Completeness check start.
Completeness check: False, reasoning: No exploration tasks have been completed, so neither major sides’ evolving strategies nor implications have been addressed, warranting foundational parallel questions.
Generated 3 next questions for exploration
Executing 3
RAG call start. Question: How have Russian military strategies evolved since the initial invasion of Ukraine in 2022?. Question context: This targets the foundational aspect of understanding Russian strategic shifts, which is essential for contextualizing the overall evolution of military strategies in the conflict.
RAG call start. Question: In what ways have Ukrainian military strategies adapted in response to changing Russian tactics throughout the war?. Question context: Investigating Ukraine’s adaptive strategies addresses a major dimension of the survey and is critical for balancing p

[FETCH]... ↓ https://www.spf.org/iina/en/articles/koizumi_02.html                                                 |
✓ | ⏱: 189.95s 

[SCRAPE].. ◆ https://www.spf.org/iina/en/articles/koizumi_02.html                                                 |
✓ | ⏱: 0.12s 

[COMPLETE] ● https://www.spf.org/iina/en/articles/koizumi_02.html                                                 |
✓ | ⏱: 190.47s 

[FETCH]... ↓ https://www.cna.org/analyses/2025/08/russian-con...re-warfare-based-on-lessons-from-the-ukraine-war  |
✓ | ⏱: 209.93s 

...

[COMPLETE] ● https://understandingwar.org/analysis/russia-ukraine/russian-offensive-campaign-assessment/          |
✓ | ⏱: 360.28s 

[FETCH]... ↓ https://www.youtube.com/watch?v=DD9vFtzDQWk                                                          |
✓ | ⏱: 360.37s 

[SCRAPE].. ◆ https://www.youtube.com/watch?v=DD9vFtzDQWk                                                          |
✓ | ⏱: 0.88s 

[COMPLETE] ● https://www.youtube.com/watch?v=DD9vFtzDQWk                                                          |
✓ | ⏱: 361.26s 

Completed iteration 6, remaining budget: 9
Completeness check start.
Completeness check: True, reasoning: All major aspects—Russian and Ukrainian strategic evolution, mutual adaptations, and future warfare implications—have been thoroughly explored with sufficient depth, directly meeting the guideline for comprehensive coverage and information gain has become low.
Literature search deemed complete by completeness checker
Survey completed with 3 responses
Starting final synthesis
Final synthesis complete
✅ Result saved to output/action_item_4_literature_search_response.json


### Step 2: Systematic Literature Search Analysis

> **Analysis Target:** `output/action_item_4_literature_search_response.json`

#### Analysis Questions

1. **Information Quality:** What information did you find most valuable? Which insights provided new understanding of the topic?
2. **Coverage Gaps:** What important aspects or perspectives appear missing from the investigation?
3. **Source Diversity:** How well did the system identify and incorporate diverse viewpoints and source types?
4. **Research Depth:** Did the investigation progress beyond surface-level information to uncover nuanced insights?

#### Algorithm Question
5. **Read the code and make comment on what's the stop criteria of this pipeline?** (hint: code entry point is `cs224v_hw1/src/literature_search.py` line 311)

---

In [ ]:
literature_search_information_quality_comment = """
Shifts in Russian and Ukrainian military strategy were the most valuable pieces of info. Namely, the sources state Russia's failure to blitzkrieg Ukraine to "win" the war, and their pivot to attrition warfare and energy weaponization (such as cutting gas supplies to Europe and conducting missile/drone strikes on Ukraine's power plants and transmission lines); this provided context in understanding the conflict from Russia's side. On the other side, the sources highlight Ukraine's drone swarm tactics, and decentralized command to shift down authority and decision-making rights down so smaller units/teams can act autonomously with shorter decision cycles and simpler objectives. They also note the set of western processes, industrial capacity, and multinational coordination that keep Ukraine resilient and with sustained resources as a defense. Moreover, it notes that AI enabled targeting systems and precision strike systems (instead of single weapons) are being used for warfare because of the improved precision, sensing, prioritization, and guidance for weapons..
"""

literature_search_coverage_gaps_comment = """
literature gaps
Some aspects were missing or underdeveloped. The analysis could have further explored humanitarian impacts, domestic political dynamics within Russia and Ukraine, and economic industrial sustainability (e.g., ammunition production, workforce fatigue). It would have been helpful to see more discussion on comparative cases (how lessons apply beyond Ukraine, like Taiwan or NATO) would have covered more.
"""

literature_search_source_diversity_comment = """
The RAG search range  pulled in a wide variety of sources coming from big research groups such as policy group RAND, Carnegie Mellon, and CSIS, and security focused institutes like CNA and GCSP, as well as economic groups like S&P Global. Overall, we think there was a good mix of policy security and economic source views. One major caveat is that these came form all western organizations that weren't directly from the Ukrainian or Russian voices, or any from non western regions. This adds some bias toward western-centric thinking. Moreover, there also wasn't any news or first hand accounts from people on the ground, so it does miss diversity in terms of global perspectives and lived experiences.  """

literature_search_research_depth_comment = """
The sources included Russia's blitzkrieg and failure to capture Kyiv in Ukraine and that being a cause of why  Russia switched to attrition and weaponized energy tactics. On the other hand in Ukraine, it explained how through drones, decentralized command units, and western multinational support has helped Ukraine's resilience. It also captured more recent news, connecting Russia’s strikes on Ukraine’s power grid using battlefield attacks to create economic and political pressure across Europe in response to losing direct economic control.
- Some limitations of the final results were that there were no details of data happening directly on the ground such as how many drones were used, succeeded, or failed, or any casualty figures.. There were also no perspectives from leaders of Ukraine or Russia. Lastly, there was no count or source of lived experiences.
"""

literature_search_stop_criteria_comment = """
Stop Criteria
- If the completeness checker in the NextStepPlanner class after aforward is ran and it returns is_complete=True, it stops.
- If the completeness checker produces no next_questions, it stops.
- If call_count >= max_retriever_calls, or in other words the retrieval budget is exceeded, it stops.

"""

with open("output/action_item_4_literature_search_response_comment.json", "w") as f:
    json.dump({
        "information_quality_comment": literature_search_information_quality_comment,
        "coverage_gaps_comment": literature_search_coverage_gaps_comment,
        "source_diversity_comment": literature_search_source_diversity_comment,
        "research_depth_comment": literature_search_research_depth_comment,
        "stop_criteria_comment": literature_search_stop_criteria_comment
    }, f, indent=2)
print(f"✅ Result saved to output/action_item_4_literature_search_response_comment.json")

✅ Result saved to output/action_item_4_literature_search_response_comment.json


## Action Item 5: Primary Source Database Exploration

> **Reference:** See **Section 4.2 (ACLED Database & Primary Source Analysis)** in the handout for comprehensive coverage of investigative research methodologies and database exploration strategies.

### Objective
Transition from literature synthesis to investigative analysis through systematic exploration of primary source data. This exercise demonstrates capabilities beyond web-accessible information by leveraging structured conflict databases.

**Data Source:** Armed Conflict Location & Event Data Project (ACLED)
- **Coverage:** Global conflict events with structured metadata
- **Temporal Scope:** Real-time updates with historical records
- **Analytical Advantages:** Systematic pattern detection, geographic clustering, temporal trend analysis

### Step 1: Database Exploration Agent Initialization

---

In [ ]:
# This dspy signature is used to generate seed questions using writeup from previous literature search to kick start the database exploration.
class ResearchQuestionGenerator(dspy.Signature):
    """You are conducting research to extract previously unknown insights by exploring and observing information in a database. Generate research questions that an investigator will be interested in. The questions will be used to generate search queries in the database to help answer them. The questions should be self-contained, meaning they must include any specific years, months, locations, or other details instead of references that require the reader to know additional context. All questions must be related to the research goal and topic. Investigate any correlations as you see fit. The questions should be completely independent of each other - if you believe some questions need to be answered first before others can be meaningful, only generate those foundational questions and not others that depend on the answers to the first questions. You do not need to generate the maximum number of questions if you believe fewer questions would be better for the research goals."""

    topic: str = dspy.InputField(desc="The research goal or topic being investigated")
    db_description: str = dspy.InputField(desc="Description of the available database including its structure, contents, and capabilities")
    max_questions: int = dspy.InputField(desc="Maximum number of questions to generate - you may generate fewer if appropriate")
    previous_insights: str = dspy.InputField(desc="Previous insights from the database")

    questions: List[str] = dspy.OutputField(desc="List of independent, self-contained research questions that will extract previously unknown insights related to the topic from the database")

generate_exploration_questions = dspy.Predict(ResearchQuestionGenerator)

In [ ]:
# research question generation
database_exploration_lm_config = LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-4.1", # database exploration planning requires high intelligence, so we use a more powerful model.
    temperature=1.0,
    max_tokens=10000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
)
database_exploration_lm = init_lm(database_exploration_lm_config)

# load the literature search response
assert os.path.exists("output/action_item_4_literature_search_response.json"), "Please run the literature search first"
literature_search_response = LiteratureSearchAgentResponse.from_dict(json.load(open("output/action_item_4_literature_search_response.json")))

with dspy.context(lm=database_exploration_lm):
    seed_questions = (await generate_exploration_questions.aforward(
        topic=TOPIC,
        db_description=ACLED_DB_DESCRIPTION,
        max_questions=4,
        previous_insights=literature_search_response.writeup
    )).questions

seed_questions_formatted = "\n\t- ".join(seed_questions)
print(f"✅ Seed questions generated. Seed questions:\n\t- {seed_questions_formatted}")

✅ Seed questions generated. Seed questions:
	- How has the frequency and geographical distribution of drone-related military events changed on both the Russian and Ukrainian sides from February 2022 to mid-2025?
	- What patterns can be observed in the use of sabotage and infiltration tactics by Russian forces behind Ukrainian lines, and how do these correlate with changes in territorial control from 2023 to mid-2025?
	- How does the integration of Western-supplied precision weapons by Ukrainian forces relate to shifts in operational outcomes, such as repelled Russian offensives and reclaimed territories, between mid-2023 and mid-2025?
	- What evidence of adaptation in command-and-control resilience and dispersed formation tactics among Ukrainian units can be observed through reported military operations and casualty rates from early 2024 to mid-2025?



> Database exploration may take up to 30-60 minutes with given topic and seed questions generated above
> **Pre-computed Results:** Available at `data/action_item_5_database_exploration_precomputed.json`  
> **Implementation Reference:** Complete code provided in Appendix section

### Step 2: Primary Source Analysis Framework

> **Reference:** See **Action Item 5 Step 2** in the handout for detailed investigative analysis methodology.

**Analysis Target:** `data/action_item_5_database_exploration_precomputed.json`

#### Key Analysis Dimensions:
1. **Database Structure and Operations:** Document which specific ACLED data tables, fields, and query patterns the agent utilized. Understanding the data architecture helps assess the comprehensiveness and reliability of findings.

2. **Quantitative Evidence Gaps:** Compare database findings with your literature search results from Action Item 4. Identify novel insights unavailable through web-based sources and assess how structured data analysis reveals patterns obscured in traditional reporting.

#### Step 3: Investigative Hypothesis Formation

Based on your comparative analysis, develop at least **2 investigative thesis statements** that synthesize insights from both literature search and database exploration. Each thesis should: be specific, highlight the tension or contradiction, name the actor and the action if presented, use concrete numbers or data if central to the story, signal impact on readers or society, and keep it concise/punchy/memorable while grounded on the facts.

---

In [ ]:
action_item_5_database_operation_comment = """
stanford_api_data is the primary table.
The fields the agent utilized in the table are:
- event_date
- civilian_targeting
- event_id_cnty
- disorder_type
- interaction
- iso
- year
- region
- country
- location
- latitude
- longitude
- source
- source_scale
- admin1
- admin2
- admin3
- event_type
- sub_event_type
- actor1
- actor2
- assoc_actor_1
- assoc_actor_2
- fatalities
- notes
- tags
- Precision codes:
--- (time_precision, geo_precision) which were put into buckets of (1=High precision, 2=Medium precision, 3=Low precision)



Query patterns:

Core filters
- Temporal filter: year BETWEN 2022 and 2024 as wel as the subset of 2024 slices
- Actor filter: Russian and Ukrainian state air components such as the `Military Forces of Russia (2000-)` and variations of this filter among different years
- Modality filters (event_type, sub_event_type): Used for drone and air strikes, shelling/missile attacks, armed clashes, and remote explosives, etc..

Aggregation & Measurement
- COUNT(*) AS event_count, SUM(fatalities) AS total_fatalities
- proportions per group via (COUNT(*) * 100 / SUM(COUNT(*)) OVER (...)
- Summary statistics over the fatalities (min, mean, median, top value histograms) used to compare the event lethality across any modality and precision levels

Precision handling of weapons were used to classify drone or air strike series of conflicts versus on-ground front line ones.
- This was done using the precision codes of (time_precision, geo_precision) which were put into the three buckets of precision levels
- Generally, it found that higher time precision meant it could classify it as an air / drone strike, whereas if medium geo precision would be classified as a front line clash.

Discovering cross border events
- For Ukrainian attributed events, the agent quantified the share of high precision air and drone operationgs as well as the fatalities per event relative to the armed events.



---
Advantages over the web narrative
- It shows that the median fatalities per drone or air event remain to be close to 0 (with a mean of ~1.46), which is not covered in the article press.
- the precision fields in ACLED make quantitative measures that air/drone ops skew high time precision while the armed clashes are fewer but more lethal in impact.
- cross-border modality -> the grouped queries allow us to clarify that the high precision airstrike or drones make up most of the activity in Ukrainian ops.

Cons to the web narrative
- the agent doesn't discuss any tactics or behind the lines pressure from all parties (both Ukraine/Russia and external influences)
- Doesn't note any air defense or interceptions like the web narrative, however ACLED proxies exist for (distrupted weapons use) or "shot down/intercepted"


literature emphasizes the attrition, drones/ai, air contestation, and border pressure, while the DB operations enable granular counts, precision stratification, and lethality distributions with quantifiable measures that the web cannot provide... This is important
because we do not know the lethality distribution or impacts of ground warfare events or air ones, keeping us uninformed about the numbers.
"""

action_item_5_database_evidence_comment = """
Drone escalation
- Air and drone events rose sharply: ~1,576 (2022)to ~2,150 (2023) to ~3,706 (2024)
- Fatalities in this subset are zero inflated.
- Shows drones are used for disruption and attrition rather than mass-casualty strikes.

Precision lethality tradeoff
- Air and drone events skew to High time precision codes, reflecting planned targeting.
- Armed clashes carry higher fatalities per event but often only medium 'geo precision'
- it quantifies a divide between precision heavy but lower lethality vs fewer and deadlier ground clashes

Cross-border modality shift
- In Belgorod/Kursk (2024), grouped queries show ~44% of Ukrainian attributed events were high precision air / drone strikes.
- These carried low fatalities per event, while armed clashes were less frequent but deadlier.
- Adds proportional clarity not available in narrative media coverage.

Geographic breadth
- ACLED data captures Russian Air Force activity outside Ukraine (e.g., Syria).
- This broader employment context is usually absent from Ukraine-centric reporting.

---
Advantages over the web narrative
- steep growth in drone strikes but consistently low lethality per event (!)
- Precision fields: allow quantitative measure of targeting quality vs lethality which literature only discusses qualitatively.
- Cross border proportions: clear evidence that Ukrainian operations leaned on high precision drone/air actions with low casualty impact.

Cons to the web narrative
- No explicit partisan/sabotage series which would require filtering by actor/notes for “partisan/resistance/sabotage.”
- No PMC vs regular analysis; Wagner and other irregular units need separate actor whitelists to compare lethality/territorial outcomes.
- Air defense intercepts underexplored; ACLED proxies exist, instead of `Disrupted weapons use`, it notes “shot down/intercepted” but weren’t extracted.

Literature emphasizes attrition, contested air, and technological adaptation.
The database adds granular counts, precision stratification, and lethality distributions that web sources lack.
This is important because it quantifies whether increased strike activity translates into higher casualties — something the narrative press rarely measures systematically.

"""

action_item_5_proposed_theses = """
1.
   Between 2022 and 2024, drone and airstrike events more than doubled (~1,576 to ~3,706),
   yet the median fatalities per event stayed near zero (mean was about 1.46).
   Contrary to the mainstream media narratives that frame drones primarily as mass casualty weapons,
   the Ukraine-Russia war points to a shift of doctrine to relentless attrition, infrastructure disruption - both economical and physical, and tempo pressure.

2.
   In Belgorod and Kursk in 2024, ~44% of Ukrainian attributed events were high precision drone/air strikes,
   producing low fatalities per event, while armed clashes were less frequent but far deadlier.
   This highlights a deliberate strategy of extending reach into Russian territory while minimizing civilian
   or troop casualties—underscoring a “reach without slaughter” model of cross border warfare.

3.
   If partisan sabotage operations in occupied Donetsk, Luhansk, Zaporizhia, Kherson, and Crimea
   are isolated through ACLED actor/notes fields, we expect a 2023 to 2024 uptick in low fatality,
   high impact attacks (IEDs, rail sabotage, arson). This would further align with the broader trend toward
   precision disruptions observed in frontline and cross border events; however this remains under quantified
   in both ACLED summaries and media coverage.
"""


with open("output/action_item_5_comments.json", "w") as f:
    json.dump({
        "action_item_5_database_operation_comment": action_item_5_database_operation_comment,
        "action_item_5_database_evidence_comment": action_item_5_database_evidence_comment,
        "action_item_5_proposed_theses": action_item_5_proposed_theses
    }, f, indent=2)

print(f"✅ Result saved to output/action_item_5_comments.json")


✅ Result saved to output/action_item_5_comments.json


## Action Item 6: DSPy Implementation for Automated Thesis Generation

> **Reference:** See **Section 4.3 (DSPy Framework)** in the handout.

### Objective
Implement a LLM thesis generation mechanism using the DSPy framework to automate the investigative hypothesis development process demonstrated manually in Action Item 5.

### Implementation Requirements

**DSPy Signature Specification:**
```python
class ThesisGenerator(dspy.Signature):
    # Implementation required
```

**Technical Specifications:**
- **Input Fields:** Research topic and database exploration insights
- **Output Specification:** Exactly 5 thesis statements
- **Constraint Requirements:** Each thesis must be specific, evidence-based, and analytically defensible
---

In [ ]:
class ThesisGenerator(dspy.Signature):
    """Generate investigative thesis statements from comprehensive research inputs.

    CORE FUNCTION: Transform literature research and database exploration findings into
    specific, defensible thesis statements that offer unique analytical perspectives
    on the research topic.

    THESIS QUALITY CRITERIA:
    - SPECIFICITY: Each thesis must make a precise, testable claim rather than broad generalizations
    - DEFENSIBILITY: Claims must be supported by evidence from the provided research inputs
    - UNIQUENESS: Each thesis should offer a distinct analytical angle or perspective
    - INVESTIGATIVE NATURE: Theses should reveal insights, patterns, or connections not immediately obvious

    GENERATION GUIDELINES:
    1. EVIDENCE INTEGRATION: Draw directly from literature summary and database findings
    2. ANALYTICAL DEPTH: Go beyond surface-level observations to identify underlying patterns
    3. DIVERSE PERSPECTIVES: Ensure the five theses explore different dimensions of the topic
    4. CITATION READINESS: Frame theses to clearly reference supporting evidence
    5. RESEARCH ADVANCEMENT: Each thesis should contribute to advancing understanding of the topic

    OUTPUT REQUIREMENTS:
    - Generate exactly 5 distinct thesis statements
    - Each thesis should be 1-2 sentences maximum
    - Implicit reference to supporting evidence without explicit citation markers
    - Avoid overlap between thesis statements
    - Prioritize insights that synthesize multiple sources
    """

    original_topic: str = dspy.InputField(
        desc="The initial research topic that was explored"
    )


    # ===============================================
    # additional input fields here
    # Hint1: how to pass insights from database exploration agent to the thesis generator?
    # Hint2: will directly concatenate question and answers from database exploration agent work? Do we need any formatting?
    # ===============================================
    literature_summary: str = dspy.InputField(
      desc="Summary of literature search findings including key concepts, theories, and prior research relevant to the topic"
    )

    database_findings: str = dspy.InputField(
        desc="Structured findings from database exploration including, questions, answers, quantitative evidnece"
    )

    # Should only have one output field as defined below. Do not change the name of the output field.
    proposed_theses: List[str] = dspy.OutputField(
          desc="Exactly 5 investigative thesis statements that synthesize unique insights from database exploration, each offering a specific, defensible claim with analytical perspective"
    )

thesis_generator = dspy.Predict(ThesisGenerator)
thesis_generator_lm = init_lm(LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-4.1", # NOTE: thesis generation requires high intelligence, so we use a more powerful model.
    temperature=1.0,
    max_tokens=10000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
))

In [ ]:
# Pass insights from database exploration agent to the thesis generator
with open("data/action_item_5_database_exploration_precomputed.json", "r") as f:
    database_exploration_precomputed = json.load(f)
    database_exploration_rag_responses =  [RagResponse.from_dict(r) for r in database_exploration_precomputed["rag_service_responses"]]

# Formatting
database_findings_formatted = ""
for r in database_exploration_rag_responses:
    database_findings_formatted += f"Question: {r.question}\n"
    database_findings_formatted += f"Answer: {r.answer}\n\n"

with dspy.context(lm=thesis_generator_lm):
    generated_theses = (await thesis_generator.aforward(
        original_topic=TOPIC,
          literature_summary=literature_search_response.writeup,
          database_findings=database_findings_formatted
        # ===============================================
        # other input fields here
        # ===============================================
    )).proposed_theses
formatted_generated_theses = "\n\t-".join(generated_theses)
print(f"generated_theses:\n\t- {formatted_generated_theses}")

with open("output/action_item_6_generated_theses.json", "w") as f:
    json.dump({
        "generated_theses": generated_theses
    }, f, indent=2)
print(f"✅ Result saved to output/action_item_6_generated_theses.json")

print(f"\n\nTake a look at the output format and skim through the content. Are you satisfied with the results? Will audience be interested in your proposed theses? With doubt, it's always a good idea to revise the prompt and try again.")
if len(generated_theses) != 5:
    raise ValueError(f"❌ Please generate exactly 5 proposed theses but found {len(generated_theses)}.")

generated_theses:
	- The marked escalation, geographic diffusion, and increased precision of cross-border Ukrainian drone and artillery strikes in Russian regions such as Belgorod and Kursk in 2024 signals a strategic doctrinal shift toward sustained, multi-modal pressure operations designed primarily for disruption and psychological impact rather than large-scale kinetic gains.
	-Both Russian and Ukrainian militaries have converged on an attrition-centric strategy that prioritizes the use of unmanned systems and information warfare, with Ukraine leveraging rapid technological adaptation and Western integration to offset manpower and materiel deficits, while Russia has responded with expanded, multi-theater drone operations that include both Ukraine and extraterritorial targets.
	-Persistent and high-frequency remote strikes—especially air, drone, and missile attacks targeting Ukrainian urban centers—demonstrate a deliberate Russian campaign to degrade urban resilience and civilian mor

**Extra human-in-the-loop step**
Manually review all generated thesis and hand pick the best 2

In [ ]:
# Review the generated these and hand pick the best 2 theses.
selected_theses = ["Persistent and high-frequency remote strikes—especially air, drone, and missile attacks targeting Ukrainian urban centers—demonstrate a deliberate Russian campaign to degrade urban resilience and civilian morale, as evidenced by seasonal surges and clustering in cities like Donetsk and Kharkiv, illustrating urban-focused warfare as a central tenet of evolving military strategy","Analysis of event data reveals that while the intensity and technological sophistication of attacks have increased, the overwhelming majority result in low to moderate fatalities per incident, indicating a future trajectory where sustained operational disruption, psychological pressure, and infrastructural degradation supersede the pursuit of mass-casualty engagements as primary objectives of modern conflict."]

assert len(selected_theses) == 2, f"❌ Please select exactly 2 theses but found {len(selected_theses)}."
with open("output/action_item_6_selected_theses.json", "w") as f:
    json.dump({
        "selected_theses": selected_theses
    }, f, indent=2)
print(f"✅ Result saved to output/action_item_6_selected_theses.json")

✅ Result saved to output/action_item_6_selected_theses.json


## Action Item 7: Automated Investigative Report Synthesis

> **Reference:** See **Section 4.4 (Synthesis & Report Composition)** in the handout for comprehensive methodology on evidence synthesis and investigative narrative construction.

### Objective
Synthesize your research findings into a comprehensive investigative report using automated DSPy-based processing pipelines that demonstrate advanced research capabilities beyond traditional literature synthesis.

### Generated Report Structure:
1. **Executive Summary:** A concise overview of your key findings and their significance
2. **Sections:** Use `#` for primary sections; Use `##` for subsections that organize detailed analysis; etc.
3. **Inline Citations:** Use numbered references in square brackets (e.g., `[1]`, `[2]`, `[3]`)
4. **Bibliography:** Conclude with a reference section where each line follows the format: `[index]. URL or source description`

### Implementation Process:

**Step 1: Thesis-Specific Literature Search**

Using your generated two theses from Action Item 6, conduct targeted literature searches for each thesis. Implement a `LiteratureSearchAgentRequest` with the `guideline` parameter to focus searches on supporting evidence for each specific thesis. Set `with_synthesis=False` to collect raw `rag_responses` for later processing.

**Step 2: Key Insight Identification**

Implement a `KeyInsightIdentifier` DSPy signature to automatically extract the most important insight from each RAG response. This reduces noise and focuses on essential information for report composition. The key insight should be a concise, one-sentence summary capturing the most relevant information for each question-answer pair.

**Step 3: Report Structure and Guideline Generation**

Create a `FinalWritingGuidelineProposal` DSPy signature that uses the collected key insights to:
- Generate a unified report thesis that synthesizes your investigative findings
- Propose a structured writing guideline in bullet-point format outlining key sections and content organization
- Ensure logical flow from background through specific discoveries to implications

**Step 4: Automated Report Synthesis**

Implement a `FinalReportSynthesizer` DSPy signature that combines the report thesis, writing guidelines, and all collected evidence into a coherent investigative report. The synthesizer should:
- Merge relevant information into logically coherent narrative sections
- Preserve all original citations exactly as provided
- Eliminate redundancy while maintaining completeness
- Create smooth transitions between thematic sections
- Constrain content to provided information without external speculation

### Quality Standards:

Your final report should demonstrate investigative depth by presenting original analytical insights not readily available in existing literature, comprehensive coverage through automated integration of evidence from multiple search iterations, logical organization with clear progression from initial questions to final conclusions, and proper attribution through systematic citation preservation.

The completed report should read as a coherent investigative piece rather than a collection of separate research summaries, showcasing the power of automated synthesis in building compelling investigative narratives from distributed evidence sources.

---

In [ ]:
## After you have generated 3 theses, we'll generate another round of literature search to support your theses. (NOTE: This comment is mistaken, it should be 2 theses not 3)
rag_responses = []

for idx, thesis in enumerate(selected_theses, 1):
    # ===============================================
    # TODO: generate literature search request. Take a look at definition of LiteratureSearchAgentRequest
    # Hint: previously we only use the field `topic` and leave other fields as default. Now we need to make use of the field `guideline`. Take a look at the source code to understand how the guideline is used.
    # Hint 2: we disable the synthesis step by setting `with_synthesis=False` as we only need the rag responses.
    # ===============================================
    literature_search_response: LiteratureSearchAgentResponse = await literature_search_agent.aforward(
        LiteratureSearchAgentRequest(topic=TOPIC,
                                     guideline=f"Find evidence to support this thesis: {thesis}. Focus on data, case studies, and research finindgs that validate this claim",
                                     with_synthesis=False))

    with open(f"output/action_item_7_literature_search_response_{idx}.json", "w") as f:
        json.dump(literature_search_response.to_dict(), f, indent=2)
    print(f"✅ Result saved to output/action_item_7_literature_search_response_{idx}.json")

    rag_responses.extend(literature_search_response.rag_responses) # TODO: add the rag response to the list


Starting literature search for topic: Evolving Military Strategies in the Russia-Ukraine War and Future Implications
Completeness check start.
Completeness check: False, reasoning: No exploration has occurred yet; critical data, case studies, and strategic analysis needed to comprehensively support the thesis remain unexplored.
Generated 3 next questions for exploration
Executing 3
RAG call start. Question: What quantitative data exists on the frequency, timing, and geographic targeting of Russian air, drone, and missile strikes on Ukrainian urban centers, particularly Donetsk and Kharkiv?. Question context: Directly addresses the need for objective data showing persistent and high-frequency strikes and supports the thesis regarding seasonal surges and urban clustering.
RAG call start. Question: What documented evidence or case studies demonstrate the specific impact of these remote strikes on urban resilience and civilian morale in targeted Ukrainian cities?. Question context: Targets

[FETCH]... ↓ https://thevoicesofwar.com/64-john-spencer-on-ukraine-urban-warfare-and-lessons-learnt/              |
✓ | ⏱: 146.40s 

[SCRAPE].. ◆ https://thevoicesofwar.com/64-john-spencer-on-ukraine-urban-warfare-and-lessons-learnt/              |
✓ | ⏱: 0.60s 

[COMPLETE] ● https://thevoicesofwar.com/64-john-spencer-on-ukraine-urban-warfare-and-lessons-learnt/              |
✓ | ⏱: 147.13s 

[FETCH]... ↓ https://www.cna.org/analyses/2025/08/russian-con...re-warfare-based-on-lessons-from-the-ukraine-war  |
✓ | ⏱: 148.14s 

...

[COMPLETE] ● https://journals.lww.com/ijsgh/fulltext/2025/050...ising_war_s_impact_on_psychiatric_care_in.2.aspx  |
✓ | ⏱: 287.55s 

[FETCH]... ↓ https://www.tandfonline.com/doi/full/10.1080/13511610.2025.2464757?af=R                              |
✓ | ⏱: 287.56s 

[SCRAPE].. ◆ https://www.tandfonline.com/doi/full/10.1080/13511610.2025.2464757?af=R                              |
✓ | ⏱: 0.47s 

[COMPLETE] ● https://www.tandfonline.com/doi/full/10.1080/13511610.2025.2464757?af=R                              |
✓ | ⏱: 288.04s 

Completed iteration 10, remaining budget: 5
Completeness check start.
Completeness check: True, reasoning: All core components of the thesis—rising attack intensity and technology, predominance of low-to-moderate fatalities, the strategic shift to operational disruption and infrastructural targeting, and psychological effects—have been comprehensively supported with quantitative data, case studies, and empirical findings, fulfilling the guideline and leaving no major knowledge gaps.
Literature search deemed complete by completeness checker
Survey completed with 5 responses
✅ Result saved to output/action_item_7_literature_search_response_2.json


In [ ]:
# Identify key insight for each rag response

# The goal of key insight identification is to identify the most important insight from the RAG response. Think of it as a one sentence summary of the RAG response that is most important to the question. The reason we do this is to help us reduce the noise in the rag responses and focus on the most important information when determing the final report outline.

class KeyInsightIdentifier(dspy.Signature):
    """Extract the most important insight from RAG response for investigative report synthesis.

    CORE FUNCTION: Distill each question-answer pair into a single, actionable insight that directly
    contributes to investigative analysis and report composition.

    KEY INSIGHT CRITERIA:
    - RELEVANCE: Must directly address the research question and context
    - CONCISENESS: Exactly one sentence that captures the essential finding
    - SPECIFICITY: Include concrete data, findings, or evidence rather than general statements
    - ANALYTICAL VALUE: Should provide investigative insight, not just factual summary
    - ACTIONABILITY: Insight should be useful for building investigative arguments

    EXTRACTION GUIDELINES:
    1. PRIORITIZE: Focus on the most significant finding that advances understanding
    2. SYNTHESIZE: Combine multiple data points into a coherent insight if needed
    3. QUANTIFY: Include specific numbers, dates, or measurements when available
    4. CONTEXTUALIZE: Frame the insight within the broader investigative context
    5. AVOID: Generic statements, obvious facts, or purely descriptive content

    OUTPUT REQUIREMENTS:
    - Single sentence format (no multiple sentences)
    - Include specific evidence or data points when available
    - Focus on investigative significance rather than surface-level information
    - Ensure insight directly supports thesis development or analysis
    """
    question: str = dspy.InputField(
        desc="The question that was asked"
    )
    question_context: str = dspy.InputField(
        desc="The context of the question"
    )
    answer: str = dspy.InputField(
        desc="The answer to the question aggregating information from external sources"
    )
    key_insight: str = dspy.OutputField(
        desc="a concise, one-sentence summary capturing the most relevant information for each question-answer pair."
    )

key_insight_identifier = dspy.Predict(KeyInsightIdentifier)
key_insight_identifier_lm = init_lm(LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-4.1-mini", # NOTE: should we use a more powerful model? You can play around with it.
    temperature=1.0,
    max_tokens=10000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
))


for rag_response in tqdm(rag_responses):
    with dspy.context(lm=key_insight_identifier_lm):
        rag_response.key_insight = (await key_insight_identifier.aforward(
            question=rag_response.question,
            question_context=rag_response.question_context,
            answer=rag_response.answer
        )).key_insight

with open("output/action_item_7_rag_responses_with_key_insight.json", "w") as f:
    json.dump([rag_response.to_dict() for rag_response in rag_responses], f, indent=2)

for rag_response in rag_responses:
    print(rag_response.key_insight + "\n")
print(f"✅ Result saved to output/action_item_7_rag_responses_with_key_insight.json")

100%|██████████| 10/10 [00:11<00:00,  1.13s/it]

From July 2025 through September 2025, Russian forces increased drone and missile strikes on Ukrainian urban centers, with September alone seeing a record 5,636 drone and 187 missile launches—averaging nearly 188 drones per day—and massed salvo attacks exceeding 800 munitions focused disproportionately on frontline cities like Donetsk and Kharkiv, evidencing a seasonal and geographic escalation aimed at overwhelming defenses and inflicting sustained civilian and infrastructure damage.

Documented case studies from Kyiv, Kharkiv, Donbas, and other cities show Russian remote strikes have substantially degraded urban resilience by destroying critical infrastructure and inflicted severe psychological stress and humanitarian crises that have significantly eroded civilian morale during prolonged urban warfare from 2022 through 2025.

Expert analyses reveal that despite acknowledging urban warfare’s critical role, Russian strategic doctrine in Ukraine remains entrenched in destructive siege t

In [ ]:
# Final report title and guideline proposal

# We will generate title and guideline for the final report. The guideline should be in bullet point format outling the key points that should be included in the report.

class FinalWritingGuidelineProposal(dspy.Signature):
    """Generate a unified report thesis and structured writing guideline from research insights.

    CORE FUNCTION: Use the collected key insights to:
    1. Generate a unified report thesis that synthesizes your investigative findings
    2. Propose a structured writing guideline in bullet-point format outlining key sections and content organization
    3. Ensure logical flow from background through specific discoveries to implications

    REPORT STRUCTURE REQUIREMENTS:
    - # Executive Summary: A concise overview of your key findings and their significance
    - # Sections: Use # for primary sections; Use ## for subsections that organize detailed analysis
    - Inline Citations: Use numbered references in square brackets (e.g., [1], [2], [3])

    GUIDELINE ORGANIZATION:
    - Ensure logical flow from background through specific discoveries to implications
    - Cover all significant themes from the key insights
    - Propose section titles using # and ## markdown formatting
    - Note where inline citations should be integrated

    OUTPUT FORMAT:
    - Report thesis: Single headline (8-14 words)
    - Writing guideline: Bullet-point format with section structure
    """
    ... # TODO: define any input fields. Hint: you can use the key insight from each rag response to help you decide what to include in the guideline.
    originial_topic: str = dspy.InputField(
        desc="The original research topic being investigated."
    )

    selected_theses: str = dspy.InputField(
        desc="The theses statements that guided evidence collection, formatted as a list."
    )

    key_insights: str = dspy.InputField(
        desc="All key insights extracted from RAG response, each representing the signigicant insight from the research."
    )
    report_thesis: str = dspy.OutputField(
        desc="A unified report thesis that synthesizes investigative findings that is exactly one headline-style line (8-14 words)") # TODO: add/improve instructions
    writing_guideline: str = dspy.OutputField(
        desc="A structured writing guideline in bullet-point format outlining key sections and content organization. Ensure logical flow from background through specific discoveries to implications. Include Executive Summary, primary sections (#), subsections (##), inline citation notes. Do NOT include bibliography/reference section." # TODO: add/improve instructions
    )

final_writing_guideline_proposal = dspy.Predict(FinalWritingGuidelineProposal)
final_writing_guideline_proposal_lm = init_lm(LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-4.1", # NOTE: report title proposal requires high intelligence, so we use a more powerful model.
    temperature=1.0,
    max_tokens=10000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
))

# Format inputs for the guideline proposal
#Selected thesis into a single input
selected_theses_formatted = "\n".join([f"{i+1}. {thesis}" for i, thesis in enumerate(selected_theses)])
#Key insights into a formatted string
key_insights_formatted = "\n".join([f"- {rag_response.key_insight}" for rag_response in rag_responses])

# TODO: add other input fields
with dspy.context(lm=final_writing_guideline_proposal_lm):
    final_writing_guideline_proposal_response = (await final_writing_guideline_proposal.aforward(
      originial_topic=TOPIC,
      selected_theses=selected_theses_formatted,
      key_insights=key_insights_formatted,
    ))

final_writing_thesis = final_writing_guideline_proposal_response.report_thesis
final_writing_guideline = final_writing_guideline_proposal_response.writing_guideline

with open("output/action_item_7_final_writing_guideline_proposal.json", "w") as f:
    json.dump({
        "report_thesis": final_writing_thesis,
        "writing_guideline": final_writing_guideline
    }, f, indent=2)

print(f"report title: {final_writing_thesis}\n\n")
print(f"writing guideline:\n {final_writing_guideline}")
print(f"✅ Result saved to output/action_item_7_final_writing_guideline_proposal.json")


report title: Remote Strikes Redefine Urban Warfare: Operational Disruption and Civilian Impact in Russia-Ukraine War


writing guideline:
 - # Executive Summary
    - Summarize the shift in Russian military strategy towards high-frequency remote strikes on Ukrainian urban centers
    - Emphasize the dual objectives of operational disruption and psychological impact over mass-casualty events [1],[2]
    - Briefly outline key evidence and implications for future warfare

- # Introduction
    - Context: Brief overview of the Russia-Ukraine War since 2022
    - Define "urban-focused warfare" and its centrality in evolving military strategies [1]
    - Introduce the research questions and significance of the study

- # Escalation of Remote and Urban-Targeted Strikes
    - ## Seasonal and Geographic Intensification
        - Data on escalation and clustering of attacks (e.g., record drone and missile launches in September 2025, frontline cities as primary targets) [1],[6]
        - Explain 

In [ ]:
def _normalize_rag_response_citation_indices(rag_responses: List[RagResponse]) -> Tuple[List[str], List[RetrievedDocument]]:
        """
        Normalize citation indices across multiple RAG (retrieval-augmented generation) responses.

        Each `RagResponse` contains:
        - `answer`: a string with inline citations like [1], [2], ...
        - `cited_documents`: the list of documents those citations refer to

        Problem:
        Citation indices restart at [1] for every response, but when combining answers,
        we want all citations to point to a single global list of retrieved documents.

        What this function does:
        1. Iterates over all RAG responses in order.
        2. Shifts the local citation indices in each answer so that they correctly map
            into the combined list of all retrieved documents.
            - For example, if the first response cited 3 docs ([1], [2], [3]),
            then the second response’s citations start at [4], not [1].
        3. Prefixes each updated answer with its corresponding sub-question for clarity.
        4. Returns:
            - A list of normalized answers (with corrected citation indices).
            - The flattened list of all retrieved documents in the proper order.

        Example:
            Input (two RAG responses):
                R1: "Paris is in France [1].", docs=[docA]
                R2: "Berlin is in Germany [1].", docs=[docB]

            Output:
                answers = [
                "Sub-question: ...\nAnswer: Paris is in France [1].",
                "Sub-question: ...\nAnswer: Berlin is in Germany [2]."
                ]
                documents = [docA, docB]
        """
        all_documents: List[RetrievedDocument] = []
        all_updated_answers: List[str] = []
        for idx, rag_response in enumerate(rag_responses):
            citation_offset = len(all_documents)
            updated_answer = rag_response.answer
            for i in range(len(rag_response.cited_documents)):
                updated_answer = updated_answer.replace(f"[{i+1}]", f"[tmp_{citation_offset+i+1}]")
            for i in range(len(rag_response.cited_documents)):
                updated_answer = updated_answer.replace(f"[tmp_{citation_offset+i+1}]", f"[{citation_offset+i+1}]")

            all_updated_answers.append(
                f"Sub-question: {rag_response.question}\nAnswer: {updated_answer}\n")
            all_documents.extend(rag_response.cited_documents)
        return all_updated_answers, all_documents

In [ ]:
# Final report synthesis

# We will synthesize the final report using generated thesis, guideline, and rag responses.

class FinalReportSynthesizer(dspy.Signature):
    """
    You are an investigative journalist composing a report with given thesis, guideline, and useful information from previous literature search.

    CORE FUNCTION: Combine report thesis, writing guideline, and all collected evidence into a coherent investigative report that demonstrates
    investigative depth through originial analytical insights not readily available in exisiting literature.

    ANALYTICAL SYNTHESIS REQUIREMENTS:
    - Identify patterns and connections ACROSS multiple sources that aren't stated in any single source
    - Look for temporal sequences (e.g., "Finding A in 2022[1] followed by Finding B in 2024[5] suggests evolution X")
    - Highlight contradictions or tensions between sources and explain their significance
    - Connect quantitative data from one source with qualitative examples from another to reveal insights
    - When multiple sources address related topics, combine them to generate insights beyond what each source states individually
    - Generate original interpretations by synthesizing complementary evidence from different sub-questions

    CONTENT INTEGRATION RULES:
    - Merge all relevant sub-question answers into a logically coherent narrativ
    - Use #, ##, ###, etc. to create title of sections and sub-sections.
    - Write section content as flowing narrative paragraphs with complete sentences, NOT bullet points or lists
    - Create clear thematic sections with smooth transitions between topics.
    - Eliminate redundancy while preserving all unique factual content
    - Exclude sub-questions/answers that don't contribute meaningfully to the survey topic
    - Maintain completeness - no loss of relevant information from source material
    - No title, conclusion, summary, or reference at the end of the answer

    CITATION PRESERVATION:
    - Use numbered reference in square brackets (e.g., [1], [2], [3])
    - If you see numbered citations like [1], [2], preserve them exactly - no format modifications
    - If you see non-numeric citations like [|domain.com] or [URL], do NOT use them
    - NEVER create citations in format like [|domain.com], [URL], or any non-numeric format
    - Place citations immediatley after the specific fact of claim being cited

    CONTENT CONSTRAINTS:
    - Base report STRICTLY on provided fathered information - add no external knowledge
    - Constrain content to provided information without external speculation
    - Use only facts and claims present in the source material
    """
    report_thesis: str = dspy.InputField(
        desc="The proposed unified thesis that synthesizes findings for the investigative journalism report (8-14 words, headline-style)"
    )
    writing_guideline: str = dspy.InputField(
        desc="The proposed writing guideline for the final report in bullet point format outlining section structure"
    )
    gathered_information: str = dspy.InputField(
        description="""Complete set of sub-question answers with their inline citations from previous research steps.
        Format typically includes:
        - Sub-question: [question text]
        - Answer: [detailed response with inline citations [1], [2], etc.]
        - (Repeated for multiple sub-questions)"""
    )

    report_style: str = dspy.InputField(
        desc="The desired style and tone for the investigative report."
    )

    # TODO: optionally add other input fields

    final_report: str = dspy.OutputField(
        desc="The final investigative report in markdown format with proper sections, inline citations using ONLY numbered format [1][2][3], and coherent narrative flow. Do not include title, conclusion, or bibliography."
    )

final_report_synthesizer = dspy.Predict(FinalReportSynthesizer)
final_report_synthesizer_lm = init_lm(LanguageModelProviderConfig(
    provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
    model_name="gpt-5-mini", # NOTE: final report synthesis requires consolidate information from very long context and requires high reasoning ability, so we use a more powerful model. If this takes too long, you can try a smaller model like gpt-5-mini or gpt-4.1.
    temperature=1.0,
    max_tokens=20000,
    litellm_server_config=LiteLLMServerConfig(api_key=userdata.get("LITELLM_API_KEY"), api_base=userdata.get("LITELLM_API_BASE"))
))

# TODO: complete the input fields. Read the function doc to understand what the function does.
all_updated_answers, all_documents = _normalize_rag_response_citation_indices(rag_responses)

## Formatted gathered information
gathered_information_formatted = "\n".join(all_updated_answers)

# TODO: format the gathered information input field
# TODO: optionallyadd other input fields
with dspy.context(lm=final_report_synthesizer_lm):
    final_report = (await final_report_synthesizer.aforward(
        report_thesis=final_writing_thesis,
        writing_guideline=final_writing_guideline,
        gathered_information=gathered_information_formatted,
        report_style="Comprehensive, highly accurate, and exhaustive; include every relevant detail and ensure no important information is omitted."
    )).final_report

with open("output/action_item_7_final_report_raw.md", "w") as f:
    f.write(final_report)
print(f"✅ Result saved to output/action_item_7_final_report_raw.md")

✅ Result saved to output/action_item_7_final_report_raw.md


In [ ]:
# TODO: manually add bibliography to the final report. Review the output. Adjust the prompt and rerun the report synthesis if necessary. Make sure it has title, executive summary, sections with desired inline citations, and bibliography.
bibliography = "\n\n# Bibliography\n\n"
for idx, doc in enumerate(all_documents, 1):
    bibliography += f"[{idx}]. {doc.url}\n\n"

final_report_with_bibliography = final_report + bibliography

with open("output/action_item_7_final_report.md", "w") as f:
    f.write(final_report_with_bibliography)
print(f"✅ Result saved to output/action_item_7_final_report.md")

✅ Result saved to output/action_item_7_final_report.md


**Review the generated report. Reflect on what are weaknesses and explain in detail how would you plan to improve it.**

In [ ]:
weaknesses_and_improvements = """
- It does a good job of including the data, and citiing it's sources.
- It did a good job comparing the Ukraine war to other historical examples to draw out some analysis of how it will
affect future wars in the region.
- It places the bulk of citations after the sentence/claim instead of embedded within the sentence. I tried
modifying the prompting to have it embeded the citations within the sentence, but wasn't able to get it to work.
- Does a good job of including a lot of statistics, and figures.
- It had other issues such as not doing enough analysis, or synthesization across sources, but after additional prompting
improvements this has largely been improved.
- It is a bit wordy and can be hard to follow at times, but it is still a good report.

"""

with open("output/action_item_7_weaknesses_and_improvements.md", "w") as f:
    f.write(weaknesses_and_improvements)
print(f"✅ Result saved to output/action_item_7_weaknesses_and_improvements.md")


## Submission guideline

1. Click "File" -> "Download" -> "Download .ipynb"
2. Rename downloaded file as "notebook.ipynb"
3. Drag and drop the "notebook.ipynb" under google colab content directory (i.e. 📁 icon on the left sidebar)
4. Run the create submission script below


In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc
!pip install nbconvert

In [ ]:
! python create_submission.py

## Download submission file

In [ ]:
from google.colab import files
files.download('/content/cs224v_hw1_submission.zip')

---

## Appendix (optional)

### Database Exploration Agent Implementation

> **Note:** This implementation is provided for reference purposes. Utilize the pre-computed results in Action Item 5 for efficient completion of the assignment.

---

In [ ]:
# API endpoint URL
api_url = "https://cs224v-database-agent.genie.stanford.edu/database-exploration"

# Prepare the request payload
payload = {
    "topic": TOPIC,
    "seed_questions": seed_questions,
    "lm_config": database_exploration_lm_config.to_dict()
}

# NOTE: uncomment the code below to make the request

print("Making request to database exploration endpoint... Might take up to 60 minutes")
if 'seed_questions' in locals():
    async with httpx.AsyncClient(timeout=6000.0) as client:
        r = await client.post(api_url, json=payload)
        r.raise_for_status()
        database_exploration_response = r.json()

    with open("output/action_item_5_database_exploration.json", "w") as f:
        json.dump(database_exploration_response, f, indent=2)

    print(f"✅ Result saved to output/action_item_5_database_exploration.json")
else:
    print("❌ 'seed_questions' is not defined. Please run the cell above to generate seed questions.")